<a href="https://colab.research.google.com/github/israelruiz2005/dio_etl_2/blob/main/C%C3%B3pia_de_ETL_projeto_DIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extração dos dados

In [1]:
!pip install pandera # instala a biblioteca

     |████████████████████████████████| 197 kB 3.2 MB/s 
     |████████████████████████████████| 10.9 MB 51.0 MB/s 


In [2]:
import pandas as pd
import pandera as pa # para validar datas

## Importa a base de dados já convertendo os campos do tipo data

In [3]:
valores_ausentes = ['**','***','****','*****','###!','####','NULL'] #fase transformação

In [4]:
df = pd.read_csv("/content/ocorrencia-alterada.csv", sep=";", parse_dates=['ocorrencia_dia'], dayfirst=True, na_values=valores_ausentes)
df.head(10)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,52242,52242,INCIDENTE,PORTO ALEGRE,RS,SBPA,2012-01-05,20:27:00,0
1,45331,45331,ACIDENTE,GUARULHOS,SP,SBGR,2012-01-06,13:44:00,3
2,45333,45333,ACIDENTE,VIAMÃO,RS,NaN,2012-01-06,13:00:00,0
3,45401,45401,ACIDENTE,SÃO SEBASTIÃO,SP,NaN,2012-01-06,17:00:00,0
4,45407,45407,ACIDENTE,SÃO SEPÉ,RS,NaN,2012-01-06,16:30:00,0
5,52243,52243,INCIDENTE,UBATUBA,SP,NaN,2012-01-06,14:30:00,0
6,50713,50713,INCIDENTE GRAVE,CAMPINAS,SP,SDAI,2012-01-07,18:15:00,0
7,45334,45334,INCIDENTE,BELÉM,PA,SBBE,2012-01-08,19:12:00,0
8,45391,45391,ACIDENTE,CONCEIÇÃO DAS ALAGOAS,MG,NaN,2012-01-08,16:00:00,0
9,52244,52244,INCIDENTE,UBERLÂNDIA,MG,SBUL,2012-01-08,22:13:00,0


In [5]:
schema = pa.DataFrameSchema (
    columns= {
        "codigo_ocorrencia":pa.Column(pa.Int),
        "codigo_ocorrencia2":pa.Column(pa.Int),
        "ocorrencia_classificacao":pa.Column(pa.String),	
        "ocorrencia_cidade":pa.Column(pa.String),	
        "ocorrencia_uf":pa.Column(pa.String, pa.Check.str_length(2,2), nullable=True),	
        "ocorrencia_aerodromo":pa.Column(pa.String, nullable=True),
        "ocorrencia_dia":pa.Column(pa.DateTime),
        "ocorrencia_hora":pa.Column(pa.String, pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5][0-9])(:[0-5][0-9])?$'),nullable=True),
        "total_recomendacoes":pa.Column(pa.Int)
    }
)

In [6]:
schema.validate(df)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,52242,52242,INCIDENTE,PORTO ALEGRE,RS,SBPA,2012-01-05,20:27:00,0
1,45331,45331,ACIDENTE,GUARULHOS,SP,SBGR,2012-01-06,13:44:00,3
2,45333,45333,ACIDENTE,VIAMÃO,RS,NaN,2012-01-06,13:00:00,0
3,45401,45401,ACIDENTE,SÃO SEBASTIÃO,SP,NaN,2012-01-06,17:00:00,0
4,45407,45407,ACIDENTE,SÃO SEPÉ,RS,NaN,2012-01-06,16:30:00,0
...,...,...,...,...,...,...,...,...,...
5162,80458,80458,ACIDENTE,JATAÍ,GO,NaN,2021-12-30,20:30:00,0
5163,80452,80452,ACIDENTE,MARACAÍ,SP,NaN,2021-12-31,09:30:00,0
5164,80457,80457,INCIDENTE GRAVE,NOVO HAMBURGO,RS,SSNH,2021-12-31,11:59:00,0
5165,80460,80460,INCIDENTE,CURITIBA,PR,SBBI,2021-12-31,15:12:00,0


### Pesquisa os tipos de dados do dataframe

In [7]:
df.dtypes

codigo_ocorrencia                    int64
codigo_ocorrencia2                   int64
ocorrencia_classificacao            object
ocorrencia_cidade                   object
ocorrencia_uf                       object
ocorrencia_aerodromo                object
ocorrencia_dia              datetime64[ns]
ocorrencia_hora                     object
total_recomendacoes                  int64
dtype: object

### Transforma uma coluna apenas apresentando o mês

In [8]:
df.ocorrencia_dia.dt.month

0        1
1        1
2        1
3        1
4        1
        ..
5162    12
5163    12
5164    12
5165    12
5166    12
Name: ocorrencia_dia, Length: 5167, dtype: int64

## Validadação de dados

Foi alterado no dataframe o parametro para tratar a data


*   dayfirst = True - convertendo o primeiro campo em dia e não mês.
*   Schema com padera - foi criada função para validação dos dados.



# Limpeza de dados

In [9]:
df.set_index('codigo_ocorrencia', inplace=True)


In [10]:
df.head()

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
codigo_ocorrencia,,,,,,,,
52242,52242,INCIDENTE,PORTO ALEGRE,RS,SBPA,2012-01-05,20:27:00,0
45331,45331,ACIDENTE,GUARULHOS,SP,SBGR,2012-01-06,13:44:00,3
45333,45333,ACIDENTE,VIAMÃO,RS,NaN,2012-01-06,13:00:00,0
45401,45401,ACIDENTE,SÃO SEBASTIÃO,SP,NaN,2012-01-06,17:00:00,0
45407,45407,ACIDENTE,SÃO SEPÉ,RS,NaN,2012-01-06,16:30:00,0


## Limpezas a realizar


*   ocorrencia_uf -> **
*   ocorrencia_aerodromo -> ###!, ####, **** e *****
*   ocorrencia_hora -> NULL




In [ ]:
# df.replace(['**','***','****','###!','####','NULL'], pd.NA, inplace=True)

In [11]:
df.head()

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
codigo_ocorrencia,,,,,,,,
52242,52242,INCIDENTE,PORTO ALEGRE,RS,SBPA,2012-01-05,20:27:00,0
45331,45331,ACIDENTE,GUARULHOS,SP,SBGR,2012-01-06,13:44:00,3
45333,45333,ACIDENTE,VIAMÃO,RS,NaN,2012-01-06,13:00:00,0
45401,45401,ACIDENTE,SÃO SEBASTIÃO,SP,NaN,2012-01-06,17:00:00,0
45407,45407,ACIDENTE,SÃO SEPÉ,RS,NaN,2012-01-06,16:30:00,0


In [12]:
df.isna().sum()

codigo_ocorrencia2             0
ocorrencia_classificacao       0
ocorrencia_cidade              0
ocorrencia_uf                  2
ocorrencia_aerodromo        1905
ocorrencia_dia                 0
ocorrencia_hora                1
total_recomendacoes            0
dtype: int64

# Trasnformação

Altera a leitura dos dados acrecentando a limpeza dos dados 


In [13]:
# filtrando
filtro = df.ocorrencia_uf.isnull()
df.loc[filtro]

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
codigo_ocorrencia,,,,,,,,
49474,49474,ACIDENTE,ÁGUAS INTERNACIONAIS,NaN,NaN,2013-09-02,02:54:00,0
79956,79956,ACIDENTE,NÃO IDENTIFICADA,NaN,NaN,2021-02-26,11:00:00,0


In [14]:
filtro = df.ocorrencia_aerodromo.isnull()
df.loc[filtro]

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
codigo_ocorrencia,,,,,,,,
45333,45333,ACIDENTE,VIAMÃO,RS,NaN,2012-01-06,13:00:00,0
45401,45401,ACIDENTE,SÃO SEBASTIÃO,SP,NaN,2012-01-06,17:00:00,0
45407,45407,ACIDENTE,SÃO SEPÉ,RS,NaN,2012-01-06,16:30:00,0
52243,52243,INCIDENTE,UBATUBA,SP,NaN,2012-01-06,14:30:00,0
45391,45391,ACIDENTE,CONCEIÇÃO DAS ALAGOAS,MG,NaN,2012-01-08,16:00:00,0
...,...,...,...,...,...,...,...,...
80446,80446,ACIDENTE,JACAREACANGA,PA,NaN,2021-12-20,17:10:00,0
80450,80450,ACIDENTE,DOURADOS,MS,NaN,2021-12-28,10:30:00,0
80455,80455,INCIDENTE GRAVE,SANTA RITA,PB,NaN,2021-12-29,18:50:00,0


In [15]:
df.count()

codigo_ocorrencia2          5167
ocorrencia_classificacao    5167
ocorrencia_cidade           5167
ocorrencia_uf               5165
ocorrencia_aerodromo        3262
ocorrencia_dia              5167
ocorrencia_hora             5166
total_recomendacoes         5167
dtype: int64

In [16]:
# ocorrencias com mais de 10 recomendaçoes
filtro= df.total_recomendacoes >10
df.loc[filtro,['ocorrencia_cidade','total_recomendacoes']]

,ocorrencia_cidade,total_recomendacoes
codigo_ocorrencia,,
47938,BRASÍLIA,11
52265,SANTOS,13
66432,VITÓRIA,12


In [17]:
# ocorrencias com classificaçao == INCIDENTE GRAVE
filtro= df.ocorrencia_classificacao == 'INCIDENTE GRAVE'
df.loc[filtro,['ocorrencia_cidade','total_recomendacoes']]

,ocorrencia_cidade,total_recomendacoes
codigo_ocorrencia,,
50713,CAMPINAS,0
45396,MARABÁ,0
50714,CAMOCIM,0
45399,MUANÁ,0
45582,RONDONÓPOLIS,0
...,...,...
80427,NOVA BANDEIRANTES,0
80448,FORMOSO DO ARAGUAIA,0
80454,ARAÇATUBA,0


In [18]:
# ocorrencias com classificaçao == INCIDENTE GRAVE  e ocorrencia_uf == 'SP' e & ou |
filtro1= df.ocorrencia_classificacao == 'INCIDENTE GRAVE'
filtro2 = df.ocorrencia_uf == 'SP'
df.loc[filtro1 & filtro2,['ocorrencia_cidade','total_recomendacoes']]

,ocorrencia_cidade,total_recomendacoes
codigo_ocorrencia,,
50713,CAMPINAS,0
45598,BRAGANÇA PAULISTA,0
45653,GUARULHOS,0
45599,BRAGANÇA PAULISTA,0
45651,ITU,1
...,...,...
80200,BRAGANÇA PAULISTA,0
80238,VOTUPORANGA,0
80265,SÃO PAULO,0


In [19]:
# ocorrencias com classificaçao == INCIDENTE GRAVE  ou INCIDENTE e ocorrencia_uf == 'SP' e & ou |
filtro1= (df.ocorrencia_classificacao == 'INCIDENTE GRAVE') | (df.ocorrencia_classificacao == 'INCIDENTE')
filtro2 = df.ocorrencia_uf == 'SP'
df.loc[filtro1 & filtro2,['ocorrencia_cidade','total_recomendacoes','ocorrencia_classificacao']]

,ocorrencia_cidade,total_recomendacoes,ocorrencia_classificacao
codigo_ocorrencia,,,
52243,UBATUBA,0,INCIDENTE
50713,CAMPINAS,0,INCIDENTE GRAVE
52248,SÃO PAULO,0,INCIDENTE
52054,SÃO PAULO,0,INCIDENTE
52055,RIBEIRÃO PRETO,0,INCIDENTE
...,...,...,...
80425,SÃO PAULO,0,INCIDENTE
80431,SANTOS,0,INCIDENTE
80453,CAMPINAS,0,INCIDENTE


In [20]:
# ocorrencias com classificaçao == INCIDENTE GRAVE  ou INCIDENTE e ocorrencia_uf == 'SP' e & ou |
filtro1= df.ocorrencia_classificacao.isin(['INCIDENTE GRAVE','INCIDENTE'])
filtro2 = df.ocorrencia_uf == 'SP'
df.loc[filtro1 & filtro2,['ocorrencia_cidade','total_recomendacoes','ocorrencia_classificacao']]

,ocorrencia_cidade,total_recomendacoes,ocorrencia_classificacao
codigo_ocorrencia,,,
52243,UBATUBA,0,INCIDENTE
50713,CAMPINAS,0,INCIDENTE GRAVE
52248,SÃO PAULO,0,INCIDENTE
52054,SÃO PAULO,0,INCIDENTE
52055,RIBEIRÃO PRETO,0,INCIDENTE
...,...,...,...
80425,SÃO PAULO,0,INCIDENTE
80431,SANTOS,0,INCIDENTE
80453,CAMPINAS,0,INCIDENTE


In [23]:
# ocorrencias DO ano 2021
filtro1= df.ocorrencia_dia.dt.year == 2021
filtro2 = df.ocorrencia_dia.dt.month == 12
filtro3 = (df.ocorrencia_dia.dt.day > 2) & (df.ocorrencia_dia.dt.day < 9)
df.loc[filtro1 & filtro2 & filtro3,['ocorrencia_cidade','ocorrencia_dia','total_recomendacoes','ocorrencia_classificacao']]

,ocorrencia_cidade,ocorrencia_dia,total_recomendacoes,ocorrencia_classificacao
codigo_ocorrencia,,,,
80415,SÃO FÉLIX DO XINGU,2021-12-03,0,ACIDENTE
80416,MANAUS,2021-12-03,0,INCIDENTE
80417,SANTO ANTÔNIO DO AMPARO,2021-12-03,0,ACIDENTE
80412,CURITIBA,2021-12-05,0,INCIDENTE
80419,LAGOA DA CONFUSÃO,2021-12-05,0,ACIDENTE
80418,VERÍSSIMO,2021-12-07,0,ACIDENTE
80424,SÃO JOSÉ DO RIO PRETO,2021-12-07,0,INCIDENTE
80433,CAMPINAS,2021-12-07,0,INCIDENTE
80420,CURITIBA,2021-12-08,0,INCIDENTE


In [24]:
# Criando nova coluna
df['ocorrencia_dia_hora'] = pd.to_datetime(df.ocorrencia_dia.astype(str) + ' ' + df.ocorrencia_hora)

In [25]:
df.head()

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
codigo_ocorrencia,,,,,,,,,
52242,52242,INCIDENTE,PORTO ALEGRE,RS,SBPA,2012-01-05,20:27:00,0,2012-01-05 20:27:00
45331,45331,ACIDENTE,GUARULHOS,SP,SBGR,2012-01-06,13:44:00,3,2012-01-06 13:44:00
45333,45333,ACIDENTE,VIAMÃO,RS,NaN,2012-01-06,13:00:00,0,2012-01-06 13:00:00
45401,45401,ACIDENTE,SÃO SEBASTIÃO,SP,NaN,2012-01-06,17:00:00,0,2012-01-06 17:00:00
45407,45407,ACIDENTE,SÃO SEPÉ,RS,NaN,2012-01-06,16:30:00,0,2012-01-06 16:30:00


In [26]:
# ocorrencias DO ano 2021
filtro_ano= df.ocorrencia_dia_hora.dt.year == 2021
filtro_mes = df.ocorrencia_dia_hora.dt.month == 12
filtro_dia = (df.ocorrencia_dia_hora.dt.day > 2) & (df.ocorrencia_dia_hora.dt.day < 9)
df.loc[filtro_ano & filtro_mes & filtro_dia,['ocorrencia_cidade','ocorrencia_dia_hora','total_recomendacoes','ocorrencia_classificacao']]

,ocorrencia_cidade,ocorrencia_dia_hora,total_recomendacoes,ocorrencia_classificacao
codigo_ocorrencia,,,,
80415,SÃO FÉLIX DO XINGU,2021-12-03 20:00:00,0,ACIDENTE
80416,MANAUS,2021-12-03 04:00:00,0,INCIDENTE
80417,SANTO ANTÔNIO DO AMPARO,2021-12-03 11:00:00,0,ACIDENTE
80412,CURITIBA,2021-12-05 12:00:00,0,INCIDENTE
80419,LAGOA DA CONFUSÃO,2021-12-05 17:55:00,0,ACIDENTE
80418,VERÍSSIMO,2021-12-07 11:10:00,0,ACIDENTE
80424,SÃO JOSÉ DO RIO PRETO,2021-12-07 19:51:00,0,INCIDENTE
80433,CAMPINAS,2021-12-07 22:00:00,0,INCIDENTE
80420,CURITIBA,2021-12-08 13:16:00,0,INCIDENTE


In [29]:
# ocorrencias DO ano 2021
filtro_ano_mes_dia_hora_inicial = df.ocorrencia_dia_hora >= '2021-12-03 11:00:00'
filtro_ano_mes_dia_hora_final = df.ocorrencia_dia_hora <= '2021-12-08 13:00:00'

df.loc[filtro_ano_mes_dia_hora_inicial & filtro_ano_mes_dia_hora_final,['ocorrencia_cidade','ocorrencia_dia_hora','total_recomendacoes','ocorrencia_classificacao']]

,ocorrencia_cidade,ocorrencia_dia_hora,total_recomendacoes,ocorrencia_classificacao
codigo_ocorrencia,,,,
80415,SÃO FÉLIX DO XINGU,2021-12-03 20:00:00,0,ACIDENTE
80417,SANTO ANTÔNIO DO AMPARO,2021-12-03 11:00:00,0,ACIDENTE
80412,CURITIBA,2021-12-05 12:00:00,0,INCIDENTE
80419,LAGOA DA CONFUSÃO,2021-12-05 17:55:00,0,ACIDENTE
80418,VERÍSSIMO,2021-12-07 11:10:00,0,ACIDENTE
80424,SÃO JOSÉ DO RIO PRETO,2021-12-07 19:51:00,0,INCIDENTE
80433,CAMPINAS,2021-12-07 22:00:00,0,INCIDENTE
80423,BRASNORTE,2021-12-08 12:45:00,0,ACIDENTE
80442,ARAL MOREIRA,2021-12-08 00:00:00,0,ACIDENTE


# agrupamentos

In [30]:
# ocorrencias DO ano 2021
filtro1= df.ocorrencia_dia.dt.year == 2021
filtro2 = df.ocorrencia_dia.dt.month == 12

df_2021_12 = df.loc[filtro1 & filtro2]

In [31]:
df_2021_12.head()

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
codigo_ocorrencia,,,,,,,,,
80413,80413,INCIDENTE GRAVE,RIBAS DO RIO PARDO,MS,SSWF,2021-12-01,10:30:00,0,2021-12-01 10:30:00
80422,80422,INCIDENTE,BELO HORIZONTE,MG,SBCF,2021-12-01,23:05:00,0,2021-12-01 23:05:00
80410,80410,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2021-12-02,09:20:00,0,2021-12-02 09:20:00
80411,80411,INCIDENTE GRAVE,LAVRAS,MG,SSOL,2021-12-02,12:15:00,0,2021-12-02 12:15:00
80414,80414,INCIDENTE,FLORES DA CUNHA,RS,SIFQ,2021-12-02,18:00:00,0,2021-12-02 18:00:00


In [32]:
df_2021_12.count()

codigo_ocorrencia2          50
ocorrencia_classificacao    50
ocorrencia_cidade           50
ocorrencia_uf               50
ocorrencia_aerodromo        31
ocorrencia_dia              50
ocorrencia_hora             50
total_recomendacoes         50
ocorrencia_dia_hora         50
dtype: int64

In [35]:
df_2021_12.groupby(['ocorrencia_classificacao']).codigo_ocorrencia2.count()

ocorrencia_classificacao
ACIDENTE           18
INCIDENTE          25
INCIDENTE GRAVE     7
Name: codigo_ocorrencia2, dtype: int64

## Agrupamento com falha, usada coluna com campo nulo

In [36]:
df_2021_12.groupby(['ocorrencia_classificacao']).ocorrencia_aerodromo.count()

ocorrencia_classificacao
ACIDENTE            2
INCIDENTE          25
INCIDENTE GRAVE     4
Name: ocorrencia_aerodromo, dtype: int64

## Contando sem usar uma coluna

In [37]:
df_2021_12.groupby(['ocorrencia_classificacao']).size()

ocorrencia_classificacao
ACIDENTE           18
INCIDENTE          25
INCIDENTE GRAVE     7
dtype: int64

### ordenando 

*   Ordem crescente 
*   Ordem descrescente





In [38]:
df_2021_12.groupby(['ocorrencia_classificacao']).size().sort_values()

ocorrencia_classificacao
INCIDENTE GRAVE     7
ACIDENTE           18
INCIDENTE          25
dtype: int64

In [39]:
df_2021_12.groupby(['ocorrencia_classificacao']).size().sort_values(ascending=False)

ocorrencia_classificacao
INCIDENTE          25
ACIDENTE           18
INCIDENTE GRAVE     7
dtype: int64

# Criando um agrupamento especifico

In [75]:
# ocorrencias DO ano 2021 regiao especifica
filtro1= df.ocorrencia_dia.dt.year == 2020
filtro2 = df.ocorrencia_uf.isin(['SP','MG','ES','RJ'])

df_2021_REGIAO = df.loc[filtro1 & filtro2]
df_2021_REGIAO

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
codigo_ocorrencia,,,,,,,,,
79043,79043,ACIDENTE,CAMANDUCAIA,MG,SNEJ,2020-01-01,21:00:00,1,2020-01-01 21:00:00
79049,79049,ACIDENTE,LEME,SP,NaN,2020-01-03,21:15:00,1,2020-01-03 21:15:00
79132,79132,INCIDENTE,SÃO MATEUS,ES,NaN,2020-01-03,14:10:00,0,2020-01-03 14:10:00
79052,79052,INCIDENTE,CAMPINAS,SP,SBKP,2020-01-05,17:15:00,0,2020-01-05 17:15:00
79054,79054,INCIDENTE,CAMPINAS,SP,SBKP,2020-01-08,13:30:00,0,2020-01-08 13:30:00
...,...,...,...,...,...,...,...,...,...
79800,79800,INCIDENTE,SÃO PAULO,SP,SBMT,2020-12-28,10:15:00,0,2020-12-28 10:15:00
79824,79824,ACIDENTE,RIO PARANAÍBA,MG,SNRP,2020-12-28,17:00:00,0,2020-12-28 17:00:00
79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0,2020-12-29 19:00:00


In [76]:
# Gerando agrupamentos

df_2021_REGIAO.groupby(['ocorrencia_classificacao']).size()

ocorrencia_classificacao
ACIDENTE            50
INCIDENTE          146
INCIDENTE GRAVE     25
dtype: int64

In [77]:
# Gerando ocorrencias por UF
df_2021_REGIAO.groupby(['ocorrencia_classificacao','ocorrencia_uf','total_recomendacoes']).size()

ocorrencia_classificacao  ocorrencia_uf  total_recomendacoes
ACIDENTE                  ES             0                       2
                                         2                       1
                          MG             0                      10
                                         1                       3
                                         2                       1
                          RJ             0                       4
                          SP             0                      19
                                         1                       6
                                         2                       3
                                         5                       1
INCIDENTE                 ES             0                       4
                          MG             0                      27
                          RJ             0                      29
                          SP             0                      86
I

In [78]:
# Agrupando por cidade 
filtro = df_2021_REGIAO.ocorrencia_cidade == 'RIO DE JANEIRO'
filtro2 = df_2021_REGIAO.total_recomendacoes > 0
df_2021_REGIAO.loc[filtro & filtro1]

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
codigo_ocorrencia,,,,,,,,,
79147,79147,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-01-19,14:30:00,0,2020-01-19 14:30:00
79269,79269,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2020-02-10,02:00:00,0,2020-02-10 02:00:00
79270,79270,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2020-02-16,20:08:00,0,2020-02-16 20:08:00
79178,79178,INCIDENTE GRAVE,RIO DE JANEIRO,RJ,SBJR,2020-02-17,19:33:00,0,2020-02-17 19:33:00
79215,79215,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2020-03-04,21:30:00,0,2020-03-04 21:30:00
79252,79252,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2020-03-09,23:20:00,0,2020-03-09 23:20:00
79250,79250,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2020-03-10,10:28:00,0,2020-03-10 10:28:00
79309,79309,ACIDENTE,RIO DE JANEIRO,RJ,NaN,2020-04-17,14:30:00,0,2020-04-17 14:30:00
79363,79363,INCIDENTE GRAVE,RIO DE JANEIRO,RJ,SBJR,2020-05-27,15:16:00,0,2020-05-27 15:16:00


In [79]:
# recomendacoes por cidade regiao
filtro = df_2021_REGIAO.total_recomendacoes > 0
df_2021_REGIAO.loc[filtro].groupby(['ocorrencia_cidade']).total_recomendacoes.sum()

ocorrencia_cidade
BAURU          1
CAJATI         2
CAMANDUCAIA    1
CATANDUVA      1
COROMANDEL     1
GUARAPARI      2
ITARIRI        2
LEME           2
MACATUBA       1
PARACATU       1
PATROCÍNIO     2
RIO CLARO      5
SÃO PAULO      1
TIETÊ          2
Name: total_recomendacoes, dtype: int64

In [80]:
# recomendacoes por cidade regiao

df_2021_REGIAO.groupby(['ocorrencia_uf','ocorrencia_cidade']).total_recomendacoes.sum()

ocorrencia_uf  ocorrencia_cidade
ES             GUARAPARI            2
               JAGUARÉ              0
               SÃO MATEUS           0
               VIANA                0
               VITÓRIA              0
                                   ..
SP             TAMBAÚ               0
               TAPIRATIBA           0
               TIETÊ                2
               VOTUPORANGA          0
               ÁLVARES MACHADO      0
Name: total_recomendacoes, Length: 80, dtype: int64

In [81]:
# recomendacoes por cidade regiao

df_2021_REGIAO.groupby(['ocorrencia_uf','ocorrencia_cidade', df_2021_REGIAO.ocorrencia_dia.dt.month]).total_recomendacoes.sum()

ocorrencia_uf  ocorrencia_cidade  ocorrencia_dia
ES             GUARAPARI          2                 2
               JAGUARÉ            4                 0
               SÃO MATEUS         1                 0
               VIANA              3                 0
               VITÓRIA            6                 0
                                                   ..
SP             TAMBAÚ             4                 0
               TAPIRATIBA         9                 0
               TIETÊ              5                 2
               VOTUPORANGA        5                 0
               ÁLVARES MACHADO    7                 0
Name: total_recomendacoes, Length: 158, dtype: int64